<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании «Хоть потоп». Нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [2]:
#подключение необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [3]:
#подключение данных 
data = pd.read_csv('/datasets/insurance.csv')

In [4]:
#просмотр содержимого датасета
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
#просмотр информации о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
#выявление пустых значений
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Пропуски и пустые значения отсутствуют. Единственное необходимо привести столбцы "Возраст" и "Зарплата" к типу int вместо float.

In [7]:
#приводим к типу int
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

In [8]:
#проверка на дубликаты
data.duplicated().sum()

153

Выводы:
* Страховые выплаты - целевой признак.
* в данных есть явные дубликаты, но в данной работе они не помеха
* все типы данных приведены к int
* пропусков в данных нет

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество не изменится.

**Обоснование:**

Предсказание для новой модели будут:

$$
an = XPw
$$

Тогда вектор весов равен:

$$
w = ((XP)^T(XP))^{-1}(XP)^Ty
$$

Подставим полуичвшийся вектор в формулу предсказания модели:

$$
an = XP((XP)^T(XP))^{-1}(XP)^Ty = XP(X^TP^TXP)^{-1}X^TP^Ty
$$

Если вынести P за скобки, то получится:

$$
an = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

$$PP^{-1} = E$$ и $$P^T(P^T)^{-1} = E$$ из чего следует: $$an=X(X^TX)^{-1}X^Ty$$

Так как $$w = (X^T X)^{-1} X^T y$$ то $$an=Xw=a$$

## Алгоритм преобразования

**Алгоритм**:

* Разделим данные на выборки
* Обучим модели и вычислим метрику R2
* Произведем умножение признаков на обратимую матрицу
* Обучим снова модель и вычислим метрику R2 на новых данных
* Сравним метрики до и после изменений

**Обоснование**

Качество регрессии не изменится так как метрики так же останутся равными.

## Проверка алгоритма

* Разделение на выборки

In [9]:
#создание признаков
features = data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']

In [10]:
#создание тестового и обучающего набора данных
features_train, features_test, target_train, target_test = train_test_split(features, target,\
                                                            test_size=0.25, random_state=1515)

* Обучение модели и вычисление метрики

In [11]:
#обучение модели и вычисление метрики
model = LinearRegression()
model.fit(features_train, target_train)
R2_origin = r2_score(target_test, model.predict(features_test))
print("Метрика R2 для начального набора данных равна: ", R2_origin)

Метрика R2 для начального набора данных равна:  0.45432015712600604


Метрика R2 для начального набора данных равна:  0.45432015712600604. Теперь создадим новый набор данных.

* Создание матрицы, проверка на обратимость и умножение ее на признаки

In [13]:
#создание матрицы
matrix = np.random.normal(size=(features_train.shape[1], features_train.shape[1]))
matrix

array([[-1.10049383, -0.82556734, -1.30691215, -0.91633749],
       [ 0.07330844, -0.18231186, -0.01242647, -0.47760811],
       [ 0.25660143,  1.56019052, -0.87029614,  0.59298037],
       [-0.83278311, -0.04305686,  0.21062384, -0.85789663]])

In [14]:
#проверка на обратимость
matrix_inverse = np.linalg.inv(matrix)
matrix_inverse

array([[-0.19785325,  1.83776897,  0.08935578, -0.75002834],
       [-0.27192866,  0.21211396,  0.53684077,  0.54343041],
       [-0.4871566 , -0.36105405, -0.28550549,  0.52400607],
       [ 0.08610646, -1.88326011, -0.18377846, -0.33619354]])

In [16]:
#умножение обратимой матрицы на признаки и создание тестовой и обчающей выборок
features_train_matrix = features_train.values @ matrix
features_test_matrix = features_test.values @ matrix

* Обучение модели и вычисление метрики R2

In [18]:
#обучение модели и вычисление метрики
model_matrix= LinearRegression()
model_matrix.fit(features_train_matrix, target_train)
R2_matrix = r2_score(target_test, model_matrix.predict(features_test_matrix))
print("Метрика R2 для матричного набора данных равна: ", R2_origin)

Метрика R2 для матричного набора данных равна:  0.45432015712600604


Метрика R2 для матричного набора данных равна:  0.45432015712600604.

**Вывод:**

* Метрика R2 для начального набора данных равна: 0.45432015712600604
* Метрика R2 для матричного набора данных равна: 0.45432015712600604

**Метрики равны. Качество модели не меняется.**

## Общий вывод

* Датасет был загружен и проанализирован
* Предварительно была проведено теоритическое исследование: изменится ли качество модели, если ее признаки умножить на обратимую матрицу.
* После теория было проверена на практике с помощью разработанного алгоритма действий
* Было доказано что **качество линейной регрессии не изменится, если ее признаки умножить на обратимую матрицу**. Качетсво модели проверялось по метрике R2, которая осталась прежней после умножения.